# Model Exploration

## Table of Contents
1. [Imports](#imports)
2. [Import Data](#importData)
    1. [GVB Data](#GVBData)
    2. [Amsterdam Event Data](#EventData)
    3. [Crowdedness Data](#CrowdData)
3. [Data Preperation](#DataPrep)
    1. [Combine Datasets](#CombData)
    2. [Construct Model DataFrame](#testModel)
    3. [Split Model DataFrame into Train and Test Data](#trainTestSplit)
4. [Regression Models](#regModels)
    1. [Baseline](#regBaseModel)
    2. [Random Forrest Regressor](#regRFGModel)
    3. [Gradient Boosting Regressor](#regGBRModel)
    4. [Ada Boost Regressor](#regABRModel)
    5. [Bagging Regressor](#regBRModel)

## Imports <a name="imports"></a>

In [168]:
import pandas as pd
import json

#Train/Test section
from sklearn.model_selection import train_test_split

#Regression Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor

#Regression Metrics
from sklearn.metrics import mean_squared_error

## Import Data <a name="importData"></a>

### GVB Data <a name="GVBData"></a>

In [2]:
#Full GVB Dataset
gvb_df = pd.read_csv("../../../Data_thesis/Full_Datasets/GVB.csv")

In [3]:
#Contents
gvb_df.head()

,Date,Hour,NieuwmarktCode,NieuwmarktLat,NieuwmarktLon,NieuwmarktArrivals,NieuwmarktDepartures,NieuwezijdsCode,NieuwezijdsLat,NieuwezijdsLon,...,DamLon,DamArrivals,DamDepartures,SpuiCode,SpuiLat,SpuiLon,SpuiArrivals,SpuiDepartures,weekday,is_weekend
0,2018-01-01,100,NMT,4.901239,52.371942,11.0,340.0,5069,4.893731,52.376288,...,52.373127,0.0,0.0,5062,4.889259,52.369097,0.0,0.0,0,0
1,2018-01-01,200,NMT,4.901239,52.371942,48.0,175.0,5069,4.893731,52.376288,...,52.373127,21.0,39.0,5062,4.889259,52.369097,0.0,0.0,0,0
2,2018-01-01,300,NMT,4.901239,52.371942,10.0,137.0,5069,4.893731,52.376288,...,52.373127,13.0,48.0,5062,4.889259,52.369097,0.0,0.0,0,0
3,2018-01-01,400,NMT,4.901239,52.371942,16.0,48.0,5069,4.893731,52.376288,...,52.373127,0.0,34.0,5062,4.889259,52.369097,0.0,0.0,0,0
4,2018-01-01,500,NMT,4.901239,52.371942,17.0,56.0,5069,4.893731,52.376288,...,52.373127,0.0,20.0,5062,4.889259,52.369097,0.0,0.0,0,0


### Amsterdam Event Data <a name="EventData"></a>

In [4]:
#Full Events Dataset
events_df = pd.read_csv("../../../Data_thesis/Full_Datasets/Events.csv")

In [5]:
events_df.head()

,Date,Event,Latitude,Longtitude
0,2018-04-20,Springsnow Festival,52.372638,4.894106
1,2018-05-20,Springsnow Festival,52.372638,4.894106
2,2018-05-20,Vurige Tongen,52.410332,4.749069
3,2018-05-21,Vurige Tongen,52.410332,4.749069
4,2018-06-03,Sneakerness,52.382834,4.920456


### Crowdedness Data <a name="CrowdData"></a>

In [129]:
#Full Crowdedness Dataset
crowd_df = pd.read_csv("../../../Data_thesis/Full_Datasets/Crowdedness.csv")

In [7]:
#Only select one camera --> GAWW-02
crowd_df = crowd_df[crowd_df["Sensor"] == "GAWW-02"]

In [8]:
crowd_df.head()

,Sensor,Date,Hour,SensorLongitude,SensorLatitude,CrowdednessCount
0,GAWW-02,2018-03-11,0,4.898903,52.373786,0
1,GAWW-02,2018-03-11,1,4.898903,52.373786,0
2,GAWW-02,2018-03-11,10,4.898903,52.373786,0
3,GAWW-02,2018-03-15,4,4.898903,52.373786,39
4,GAWW-02,2018-04-21,17,4.898903,52.373786,1618


## Data Preperation <a name="DataPrep"></a>

### Combine datasets <a name="CombData"></a>

In [130]:
df = pd.merge(gvb_df, crowd_df, on=["Date", "Hour"])

In [131]:
full = pd.merge(events_df, df, on=["Date"], how="outer")
full = full.rename(index=str, columns={"Latitude": "event_lat", "Longtitude": "event_lon", "Event": "is_event"})
full = full.fillna(0.0)

In [132]:
full_dict = full.to_dict("index")

In [133]:
for k, v in full_dict.items():
    if v["is_event"] != 0.0:
        v["is_event"] = 1.0

In [134]:
full = pd.DataFrame.from_dict(full_dict, orient="index")

In [135]:
full.head()

,Date,is_event,event_lat,event_lon,Hour,NieuwmarktCode,NieuwmarktLat,NieuwmarktLon,NieuwmarktArrivals,NieuwmarktDepartures,...,SpuiLat,SpuiLon,SpuiArrivals,SpuiDepartures,weekday,is_weekend,Sensor,SensorLongitude,SensorLatitude,CrowdednessCount
0,2018-04-20,1.0,52.372638,4.894106,0.0,NMT,4.901239,52.371942,44.0,143.0,...,4.889259,52.369097,13.0,40.0,4.0,0.0,GAWW-02,4.898903,52.373786,0.0
1,2018-04-20,1.0,52.372638,4.894106,0.0,NMT,4.901239,52.371942,44.0,143.0,...,4.889259,52.369097,13.0,40.0,4.0,0.0,GAWW-02,4.898903,52.373786,747.0
10,2018-05-20,1.0,52.358560,4.897830,0.0,NMT,4.901239,52.371942,83.0,225.0,...,4.889259,52.369097,0.0,70.0,6.0,1.0,GAWW-02,4.898903,52.373786,2263.0
100,2018-06-02,1.0,52.372638,4.894106,0.0,NMT,4.901239,52.371942,42.0,67.0,...,4.889259,52.369097,0.0,0.0,5.0,1.0,GAWW-02,4.898903,52.373786,1565.0
1000,2018-07-24,1.0,52.339047,4.891153,0.0,NMT,4.901239,52.371942,49.0,134.0,...,4.889259,52.369097,0.0,28.0,1.0,0.0,GAWW-03,4.897334,52.372524,1603.0


### Construct Subset to train and test Model <a name="testModel"></a>

In [187]:
#Drop Unusable columns
model_df = full.drop(columns=["Date","NieuwmarktCode", "NieuwezijdsCode", "DamCode", "SpuiCode", "Sensor"])

In [188]:
model_df.head()

,is_event,event_lat,event_lon,Hour,NieuwmarktLat,NieuwmarktLon,NieuwmarktArrivals,NieuwmarktDepartures,NieuwezijdsLat,NieuwezijdsLon,...,DamDepartures,SpuiLat,SpuiLon,SpuiArrivals,SpuiDepartures,weekday,is_weekend,SensorLongitude,SensorLatitude,CrowdednessCount
0,1.0,52.372638,4.894106,0.0,4.901239,52.371942,44.0,143.0,4.893731,52.376288,...,312.0,4.889259,52.369097,13.0,40.0,4.0,0.0,4.898903,52.373786,0.0
1,1.0,52.372638,4.894106,0.0,4.901239,52.371942,44.0,143.0,4.893731,52.376288,...,312.0,4.889259,52.369097,13.0,40.0,4.0,0.0,4.898903,52.373786,747.0
10,1.0,52.358560,4.897830,0.0,4.901239,52.371942,83.0,225.0,4.893731,52.376288,...,456.0,4.889259,52.369097,0.0,70.0,6.0,1.0,4.898903,52.373786,2263.0
100,1.0,52.372638,4.894106,0.0,4.901239,52.371942,42.0,67.0,4.893731,52.376288,...,152.0,4.889259,52.369097,0.0,0.0,5.0,1.0,4.898903,52.373786,1565.0
1000,1.0,52.339047,4.891153,0.0,4.901239,52.371942,49.0,134.0,4.893731,52.376288,...,301.0,4.889259,52.369097,0.0,28.0,1.0,0.0,4.897334,52.372524,1603.0


### Train/Test Split <a name="trainTestSplit"></a>
Split the dataset into training and test data, using a [function from Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

#### Train_Test_split Parameters
- *input*: 
    - *x*: data used to predict laballed data
        - Hour slot
        - Weekday (int)
        - Weekend (Binary)
    - *y*: laballed data
        - Number of rides (int)
- *test_size*: Proportion of total data used as test data, rest is training data
- *random_state*: Number used as seed by the *Random Number Generator* 

In [189]:
feature_labels = model_df.columns.values

#Split the labels from the rest
x = model_df.drop(["CrowdednessCount"], axis=1)
y = model_df["CrowdednessCount"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.10, random_state = 42)

## Regression Models <a name="regModels"></a>

In [206]:
print("MSE")
print("Baseline: ", reg_mse)
print("Random Forrest Regressor: ", reg_rfg_mse)
print("Gradient Boosting Regressor: ", reg_gbr_mse)
print("Ada Boost Regressor: ", reg_abr_mse)

MSE
Baseline:  193554.88187989246
Random Forrest Regressor:  275146.98422777443
Gradient Boosting Regressor:  189046.07618334529
Ada Boost Regressor:  222777.38807823937


### Linear Regression <a name="regBaseModel"></a>
Implemented the [Sklearn Version](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)

In [190]:
#Give parameters model
reg_base = LinearRegression(n_jobs=5)

In [191]:
#Fit the model
reg_base.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=5, normalize=False)

In [192]:
reg_base_score = reg_base.score(x_test, y_test)

y_pred = reg_base.predict(x_test)
reg_mse = mean_squared_error(y_pred, y_test)

print("R^2 Score: ", reg_base_score)
print("MSE: ", reg_mse)

R^2 Score:  0.5410984512822687
MSE:  193554.88187989246


### Random Forrest Regressor <a name="regRFGModel"></a>
Implemented the [Sklearn Version](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

Parameters
- *N-Estimators*: Number of trees in the model
- *criterion*: loss function
- *n_jobs*: The number of jobs to run in parallel for both fit and predict
- *random_state*: random_state is the seed used by the random number generator
- *bootstrap*: Whether bootstrap samples are used when building trees

In [193]:
#Set parameters model
reg_rfg = RandomForestRegressor(n_estimators=500, criterion="mse", n_jobs=100, random_state=42, bootstrap=True)

In [194]:
#fit the model
reg_rfg.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=100,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [195]:
#Score the model
reg_rfg_score = reg_rfg.score(x_test, y_test)

y_pred = reg_rfg.predict(x_test)
reg_rfg_mse = mean_squared_error(y_pred, y_test)

print("R^2 Score: ", reg_rfg_score)
print("MSE: ", reg_rfg_mse)

R^2 Score:  0.34765077501123975
MSE:  275146.98422777443


In [196]:
#Feature importance
importance = reg_rfg.feature_importances_
feature_indexes_by_importance = importance.argsort()
for index in feature_indexes_by_importance:
    print('{}-{:.2f}%'.format(feature_labels[index], (importance[index] *100.0)))

Hour-0.00%
SpuiLat-0.01%
DamLat-0.01%
NieuwmarktLat-0.01%
NieuwezijdsLat-0.01%
NieuwmarktLon-0.01%
NieuwezijdsLon-0.02%
SpuiLon-0.02%
DamLon-0.02%
is_event-1.68%
SpuiArrivals-2.24%
NieuwezijdsArrivals-3.30%
DamArrivals-3.58%
SpuiDepartures-3.80%
NieuwmarktArrivals-4.09%
NieuwezijdsDepartures-4.82%
event_lat-5.94%
NieuwmarktDepartures-6.18%
event_lon-6.56%
DamDepartures-8.19%
is_weekend-10.13%
weekday-12.45%
SensorLatitude-13.42%
SensorLongitude-13.50%


### Gradient Boosting Regressor <a name="regGBRModel"></a>
Implemented the [Sklearn Version](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor)

Parameters
- *Loss*: Loss function
- *Learning_rate*: Learning rate shrinks the contribution of each tree by learning_rate
- *n_estimators*: The number of boosting stages to perform
- *Criterion*: The function to measure the quality of a split
- *n_iter_no_change*: used to decide if early stopping will be used to terminate training when validation score is not improving
- *validation_fraction*: The proportion of training data to set aside as validation set for early stopping

In [197]:
reg_gbr = GradientBoostingRegressor(loss="ls", learning_rate=0.05, n_estimators=300, criterion="friedman_mse", 
                                    random_state=42, n_iter_no_change=20, validation_fraction=0.1)

In [198]:
reg_gbr.fit(x_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=300, n_iter_no_change=20, presort='auto',
             random_state=42, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [199]:
reg_gbr_score = reg_gbr.score(x_test, y_test)

y_pred = reg_gbr.predict(x_test)
reg_gbr_mse = mean_squared_error(y_pred, y_test)

print("R^2 Score: ", reg_gbr_score)
print("MSE: ", reg_gbr_mse)

R^2 Score:  0.5517884318031256
MSE:  189046.07618334529


In [200]:
#Feature importance
importance = reg_gbr.feature_importances_
feature_indexes_by_importance = importance.argsort()
for index in feature_indexes_by_importance:
    print('{}-{:.2f}%'.format(feature_labels[index], (importance[index] *100.0)))

Hour-0.00%
NieuwmarktLon-0.00%
NieuwezijdsLat-0.00%
NieuwezijdsLon-0.00%
SpuiLon-0.00%
DamLat-0.00%
DamLon-0.00%
SpuiLat-0.00%
NieuwmarktLat-0.00%
is_event-0.83%
DamArrivals-0.85%
NieuwezijdsArrivals-1.09%
event_lon-1.29%
SpuiArrivals-1.41%
NieuwmarktArrivals-2.45%
NieuwmarktDepartures-2.68%
NieuwezijdsDepartures-2.87%
SpuiDepartures-2.99%
event_lat-5.41%
is_weekend-9.56%
DamDepartures-13.26%
SensorLongitude-16.42%
weekday-18.77%
SensorLatitude-20.14%


### Ada Boost Regressor <a name="regABRModel"></a>
Implemented the [Sklearn Version](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor)

Parameters
- *n_estimators*: The maximum number of estimators at which boosting is terminated
- *learning_rate*: Learning rate shrinks the contribution of each regressor
- *loss*: The loss function to use when updating the weights after each boosting iteration.

In [201]:
reg_abr = AdaBoostRegressor(n_estimators=100, learning_rate= 0.5, loss="square", random_state=42)

In [202]:
reg_abr.fit(x_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=0.5, loss='square',
         n_estimators=100, random_state=42)

In [203]:
reg_abr_score=reg_abr.score(x_test, y_test)

y_pred = reg_abr.predict(x_test)
reg_abr_mse = mean_squared_error(y_pred, y_test)

print("R^2 Sore: ", reg_abr_score)
print("MSE: ", reg_abr_mse)

R^2 Sore:  0.471814467217447
MSE:  222777.38807823937


In [204]:
#Feature importance
importance = reg_abr.feature_importances_
feature_indexes_by_importance = importance.argsort()
for index in feature_indexes_by_importance:
    print('{}-{:.2f}%'.format(feature_labels[index], (importance[index] *100.0)))

Hour-0.00%
NieuwmarktLat-0.00%
NieuwezijdsLat-0.00%
NieuwezijdsLon-0.00%
DamLat-0.00%
DamLon-0.00%
NieuwmarktLon-0.00%
SpuiLat-0.00%
SpuiLon-0.01%
is_event-1.14%
NieuwezijdsArrivals-1.96%
event_lon-3.21%
is_weekend-3.54%
DamArrivals-3.68%
NieuwmarktDepartures-4.03%
SpuiArrivals-4.59%
SensorLatitude-6.37%
SensorLongitude-6.57%
event_lat-7.13%
NieuwmarktArrivals-9.51%
weekday-10.09%
SpuiDepartures-10.90%
NieuwezijdsDepartures-12.46%
DamDepartures-14.81%
